<a href="https://colab.research.google.com/github/margaretxie/Test_Paper3_with_UCO/blob/main/UCO_v8_cam234_calculate_obo_mea_by_type_33KP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

#test paper 3 - the HK one - with UCO dataset (only cam234, not cam 0 1, total 1296 videos )

# Paper 3 - https://link.springer.com/article/10.1007/s13755-023-00258-3  -skeleton based repeptition count method
# The paper's code - webpage: https://github.com/YuChengHSU/repetition-counting/tree/main

# UCO dataset [paper20] - downloaded from  http://rabinf24.uco.es/avaphy/    (Outlook Emailed got access )


# v8: v4_based, 33KPs_obo, mea by type
# v7: v3_based, 33KPs_ merge_estimated_n_groundtrue_count
# v6: v2_based, apply original paper3 model (using whole 33 KPs)
# v5: average KP used, popular KPs analyst
# v4: calculate obo, mea according to action types (1-16)
# v3: merge_estimated_n_groundtrue_count
# v2: apply paper3's model to filtered KPs which visibility >=threshold
# v1: extract each videos' KPs  (this code only for half cameras (2,3,4))
# v0: framework of looping all the .mp4 files, with name PnAn (Person*Action*)

In [ ]:
from types import new_class
from google.colab import drive
import pandas as pd
import numpy as np
import os

# Mount Google Drive
drive.mount('/content/gdrive')

v="8"

# File path on Google Drive
base_path = base_path = '/content/gdrive/My Drive/repetition_dataset/UCO/results_cam234'

file_path = os.path.join(base_path, 'V7_combined_estimated_and_groundtruth_counts_33KP.csv' )
new_file_path = os.path.join(base_path, f'V{v}_OBO_MAE_33KP.csv' )

# Load the data
data = pd.read_csv(file_path)

# calculate OBO Correct Classification Rate
def calculate_obo(data):
    # Calculate OBO as the rate of correct classifications
    obo_correct = (abs(data['Estimated Count'] - data['count']) <= 1).mean()
    return obo_correct  # Higher is better (correct classification rate)

# Function to calculate MAE
def calculate_mae(data):
    conditions = data['count'] != 0
    data.loc[conditions, 'MAE'] = abs(data['Estimated Count'] - data['count']) / data['count']
    data.loc[~conditions, 'MAE'] = np.nan  # Treat division by zero cases as NaN
    return data['MAE'].mean(skipna=True)

# Calculating metrics for each type and overall
results = []
for type_ in data['type'].unique():
    type_data = data[data['type'] == type_].copy()
    obo_correct = calculate_obo(type_data)
    mae = calculate_mae(type_data)
    record_count = type_data.shape[0]
    results.append({'Type': type_, 'OBO (Higher Better)': round(obo_correct, 2), 'MAE (Lower Better)': round(mae, 2), 'Record Count': record_count})

# Convert to DataFrame
results_df = pd.DataFrame(results)

# Calculate overall metrics
overall_obo = calculate_obo(data)
overall_mae = calculate_mae(data)

# Append overall metrics to the DataFrame using pd.concat
overall_df = pd.DataFrame([{'Type': 'Overall', 'OBO (Higher Better)': round(overall_obo, 2), 'MAE (Lower Better)': round(overall_mae, 2), 'Record Count': data.shape[0]}])
results_df = pd.concat([results_df, overall_df], ignore_index=True)

# Sort the DataFrame by OBO Correct Rate in ascending order
results_df = results_df.sort_values('OBO (Higher Better)', ascending=True)

# Print the sorted DataFrame before saving

print('')
print('')
print('     V8_UCO_Paper3_Origianl(33KPs)_CAM234_OBO_MEA_by_type  f=25')
print('   ===============================================================')
print('')
print(results_df)

# Save to new CSV file
results_df.to_csv(new_file_path, index=False)



Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


     V8_UCO_Paper3_Origianl(33KPs)_CAM234_OBO_MEA_by_type  f=25

       Type  OBO (Higher Better)  MAE (Lower Better)  Record Count
13       16                 0.21                1.26            81
9        12                 0.30                1.18            81
5         8                 0.63                0.40            81
1         4                 0.64                0.41            81
7        10                 0.65                0.46            81
12       15                 0.70                0.32            81
16  Overall                 0.71                0.40          1295
2         5                 0.72                0.32            81
11       14                 0.75                0.34            81
8        11                 0.79                0.30            81
14        1                 0.80                0.23            8